In [1]:
!pip install ffmpeg

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6089 sha256=c0c7e646e7841e95ac2940d6948101db93c4a851ed94a287f9b7e318230ee8e4
  Stored in directory: c:\users\rohit\appdata\local\pip\cache\wheels\26\21\0c\c26e09dff860a9071683e279445262346e008a9a1d2142c4ad
Successfully built ffmpeg


In [2]:
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import glob
from pathlib import Path

ERROR:absl:cannot import name 'rlds' from partially initialized module 'tensorflow_datasets' (most likely due to a circular import) (C:\Users\rohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow_datasets\__init__.py)
Traceback (most recent call last):
  File "C:\Users\rohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow_datasets\__init__.py", line 79, in <module>
    from tensorflow_datasets import rlds  # pylint: disable=g-bad-import-order
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ImportError: cannot import name 'rlds' from partially initialized module 'tensorflow_datasets' (most likely due to a circular import) (C:\Users\rohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow_datasets\__init__.py)


ModuleNotFoundError: No module named 'tensorflow.python'

**SKELETON VIDEO TO REALISTIC VIDEO**

In [ ]:
# Install required package
!pip uninstall dominate
!pip install dominate --quiet

Reading the video and breaking into frames

In [ ]:
import cv2
import os, shutil
# Open the video file
merged_output = './hello.mp4'
# Open the video file
video = cv2.VideoCapture(merged_output)
path = "./frames/initial"

# Delete any previous frames in the folder
for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Check if the video file was opened successfully
if not video.isOpened():
    print("Error opening video file")
    exit()

# Set the frame number to start with
frame_num = 0

# Loop through the video frames
while True:
    # Read the next frame from the video
    ret, frame = video.read()

    # Check if the frame was read successfully
    if not ret:
        break

    # Save the frame as an image file
    # cv2.imwrite(f"frame{frame_num}.png", frame)
    cv2.imwrite(os.path.join(path , f"frame{frame_num}.png"), frame)

    # Increment the frame number
    frame_num += 1

# Release the video object
video.release()

Cropping the inferred image

In [ ]:
import os
import cv2

# Set the paths for the input folder containing images and the output folder
input_folder = './frames/initial'
output_folder = './frames/test_A'

# Delete any previous frames in the folder

for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Desired crop dimensions and offset
crop_width = 400
crop_height = 336
offset = 40

# Loop over the images in the input folder
for filename in os.listdir(input_folder):
    # Read the image
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)
    # Calculate the crop coordinates
    image_height, image_width = img.shape[:2]
    start_x = max(0, int((image_width - crop_width) / 2))
    start_y = max(0, int((image_height - crop_height) / 2)) - offset
    
    # Perform the crop
    cropped_image = img[start_y:start_y + crop_height, start_x:start_x + crop_width]
    
    # Create the output path for the cropped image
    output_path = os.path.join(output_folder, filename)
    
    # Save the cropped image
    cv2.imwrite(output_path, cropped_image)

In [ ]:
%cd /content/pix2pixHD

Running the GAN inference on the above frames

To sort the files in alphanumeric order

In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
from os.path import isfile, join

pathIn = '/content/pix2pixHD/results/pose2sign/test_latest/images'
pathOut = 'GAN_generated_new.mp4'
fps = 10
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()

In [ ]:
sorted_list = sorted_alphanumeric(files)

Creating the video from inferred frames

In [ ]:
import cv2
import fnmatch

for file in sorted_list:
  if fnmatch.fnmatch(file, '*_synthesized_image.jpg'):
    filename = pathIn +'/'+ file
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

Compressing the video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/pix2pixHD/GAN_generated_new.mp4"

# Compressed video path
compressed_path = "/content/pix2pixHD/compressed_generated_new.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

Displayin ouput video

In [ ]:
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=700 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)